In [80]:
# !pip install selenium
# !pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

In [81]:
# !pip install pyresparser

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import pickle
import nltk
import numpy as np

In [2]:

# !pip install spacy==2.3.5
# !pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

In [3]:
# !pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ankur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ankur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ankur\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ankur\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
# !python -m spacy download en

In [6]:
with open('NaiveBayes.pickle', 'rb') as efile:
    clf=pickle.load(efile)

c:\users\ankur\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator MultinomialNB from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\ankur\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LabelBinarizer from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\ankur\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator OneVsRestClassifier from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [7]:
with open('TFIDFVectorizer.pickle', 'rb') as efile:
    word_vectorizer=pickle.load(efile)

c:\users\ankur\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\ankur\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [8]:
with open('LabelEncoder.pickle', 'rb') as efile:
    le=pickle.load(efile)

c:\users\ankur\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [10]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english')) #creating a set of stop words
def cleanResume(resumeText):
  
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    word_tokens = word_tokenize(resumeText) #Tokenize words
    filtered_sentence = [w for w in word_tokens if not w in stop_words] #Contains words other than stop words
    filtered_sentence = map(lambda x: lemmatizer.lemmatize(x), filtered_sentence)
    
    resumeText=' '.join(filtered_sentence)
    return resumeText

In [11]:
from pyresparser import ResumeParser
data = ResumeParser('VithikaPungliya_Resume.pdf').get_extracted_data()

c:\users\ankur\appdata\local\programs\python\python39\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [12]:
sentences=" "
omit=["name", "email", "mobile_number", "no_of_pages", "total_experience"]
for key, value in data.items():
      if value !=None and key not in omit: 
        if type(value)==list:
              sentences= sentences +" "+ key + " " + " ".join(value)
        else:
              sentences= sentences +" "+ key + " " + value

In [13]:
def largest_indices(ary, n):
    flat = ary.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    return indices

In [14]:
def test(sent):
  sent=cleanResume(sent)
  WordFeatures = word_vectorizer.transform([sent])
  x=largest_indices(clf.predict_proba(WordFeatures), 3)
  recommendation=le.inverse_transform(x)
  return recommendation

In [15]:
professions= test(sentences)

In [16]:
professions

array(['Data Science', 'Python Developer', 'Java Developer'], dtype=object)

In [17]:
# !pip install html5lib

In [21]:
counter=0
df = pd.DataFrame(columns=['Title','Company','Ratings','Salary','Location','Job_Post_History','URL', 'Skills', "Profession"])
for i in professions:
    counter=counter+1
    for j in range(1,3):
        a=i.replace(" ","-")
        b=i.replace(" ","%20")
        url = f"https://www.naukri.com/{a}-jobs-{j}?k={b}"

        driver = webdriver.Chrome("chromedriver.exe")
        driver.get(url)
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source,'html5lib')
#         print(soup)

        driver.close()

        results = soup.find(class_='list')
        print(results)
        job_elems = results.find_all('article',class_='jobTuple bgWhite br4 mb-8')

        for job_elem in job_elems:

            # URL to apply for the job     
            URL = job_elem.find('a',class_='title fw500 ellipsis').get('href')

            # Post Title
            Title = job_elem.find('a',class_='title fw500 ellipsis')

            # Company Name
            Company = job_elem.find('a',class_='subTitle ellipsis fleft')

            # Ratings
            rating_span = job_elem.find('span',class_='starRating fleft dot')
            if rating_span is None:
                continue
            else:
                Ratings = rating_span.text


            # Salary offered for the job
            Sal = job_elem.find('li',class_='fleft grey-text br2 placeHolderLi salary')
            Sal_span = Sal.find('span',class_='ellipsis fleft fs12 lh16')
            if Sal_span is None:
                continue
            else:
                Salary = Sal_span.text

            # Location for the job post
            Loc = job_elem.find('li',class_='fleft grey-text br2 placeHolderLi location')
            Loc_exp = Loc.find('span',class_='ellipsis fleft fs12 lh16')
            if Loc_exp is None:
                continue
            else:
                Location = Loc_exp.text

            # Number of days since job posted
            Hist = job_elem.find("div",["type br2 fleft grey","type br2 fleft green"])
            Post_Hist = Hist.find('span',class_='fleft fw500')
            if Post_Hist is None:
                continue
            else:
                Post_History = Post_Hist.text
                
            #Skills
            skills=job_elem.find_all("ul", ["tags has-description"])

        #   Appending data to the DataFrame 
            df=df.append({'URL':URL,'Title':Title.text,'Company':Company.text,'Ratings':Ratings,'Salary':Salary,'Location':Location,'Job_Post_History':Post_History, "Profession":i,"Profession_key":counter, "Skills":skills},ignore_index = True)
#         print(df)

C:\Users\ankur\AppData\Local\Temp/ipykernel_3788/469758879.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


None


AttributeError: 'NoneType' object has no attribute 'find_all'

In [99]:
#Display on Website
#Every Profession highest cosine similarity 10
#Profession
#Company
#Position
#Ratings- Filter
#Location
#Job_Post_History
#URL
#Skills
#Cosine Similiarty Index

#Index
#Resume Skills and Needed/Required Skills- Cosine Similarity

In [100]:
#Build Cosine Similarity

In [103]:
df

,Title,Company,Ratings,Salary,Location,Job_Post_History,URL,Skills,Profession,Profession_key
0,Senior Business Analyst - Data Science,Freshworks,4.0,Not disclosed,Chennai,1 Day Ago,https://www.naukri.com/job-listings-senior-bus...,"[[[python], [PowerBI], [analytical], [Databric...",Data Science,1.0
1,Dataiku Data Science Studio (DSS) Application ...,Accenture,4.2,Not disclosed,Pune,1 Day Ago,https://www.naukri.com/job-listings-dataiku-da...,"[[[Business process], [MySQL], [Consulting], [...",Data Science,1.0
2,Analyst-Data Science,Accenture,4.2,Not disclosed,Bangalore/Bengaluru,1 Day Ago,https://www.naukri.com/job-listings-analyst-da...,"[[[Publishing], [Consulting], [Predictive mode...",Data Science,1.0
3,"Analyst, Data Science & Analytics",TransUnion,4.3,Not disclosed,Pune,5 Days Ago,https://www.naukri.com/job-listings-analyst-da...,"[[[SQL], [communication], [IT Skills], [Data S...",Data Science,1.0
4,Data Science Manager,Rapido,4.0,Not disclosed,Bangalore/Bengaluru,1 Day Ago,https://www.naukri.com/job-listings-data-scien...,"[[[Data Science], [data analysis], [NoSql], [A...",Data Science,1.0
...,...,...,...,...,...,...,...,...,...,...
59,Senior Software Developer - JBPM/Java,SteerLean,3.6,Not disclosed,"Kolkata, Mumbai, Visakhapatnam, Hyderabad/Secu...",Today,https://www.naukri.com/job-listings-senior-sof...,[],Java Developer,3.0
60,Sr. Java Developer / Lead,CIGNEX Datamatics,4.3,Not disclosed,"Kolkata, Pune, Chennai, Bangalore/Bengaluru, D...",3 Days Ago,https://www.naukri.com/job-listings-sr-java-de...,[],Java Developer,3.0
61,Hiring Senior java Developers/Team Leads with ...,Netcracker,3.8,Not disclosed,Pune,1 Day Ago,https://www.naukri.com/job-listings-hiring-sen...,[],Java Developer,3.0
62,Senior Java Developer,V2Soft,3.6,Not disclosed,Chennai,1 Day Ago,https://www.naukri.com/job-listings-senior-jav...,[],Java Developer,3.0


In [101]:
def convert_list(ls):
    ls=np.array(ls)
    l=[]
    
    for x in np.nditer(ls):
        l.append(str(x)) 
    return l

In [102]:
df["Skills"]=df["Skills"].apply(convert_list)

ValueError: Iteration of zero-sized operands is not enabled

In [104]:
df["Skills"]

0     [[[python], [PowerBI], [analytical], [Databric...
1     [[[Business process], [MySQL], [Consulting], [...
2     [[[Publishing], [Consulting], [Predictive mode...
3     [[[SQL], [communication], [IT Skills], [Data S...
4     [[[Data Science], [data analysis], [NoSql], [A...
                            ...                        
59                                                   []
60                                                   []
61                                                   []
62                                                   []
63                                                   []
Name: Skills, Length: 64, dtype: object

In [ ]:
user_skills=" ".join(data["skills"])
user_skills

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
cv = CountVectorizer()
def score(ls):
    JobText=' '.join(ls)
    text=[user_skills,JobText]
    count_matrix = cv.fit_transform(text)
    matchPercentage=cosine_similarity(count_matrix)[0][1] * 100
    return round(matchPercentage, 2)
    

In [ ]:
df["score"]=df["Skills"].apply(score)

In [ ]:
top15=df.sort_values('score',ascending=False)[:15]

In [ ]:
top15

In [ ]:
top5_2 = df_sorted[df_sorted['Profession_key'] == 2.0 ][:5]

In [ ]:
top5_2

In [ ]:
top5_3=df_sorted[df_sorted['Profession_key'] == 3.0 ][:5]

In [ ]:
top5_3